In [1]:
import utils
import loader
import torch
path_HE = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/HE'
path_IHC = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/IHC'

params = utils.get_config_from_yaml('C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/torch_stain_transfer/code/config.yaml')
k=1

train_data = loader.stain_transfer_dataset( img_patch=  k,
                                            params= params,
                                            HE_img_dir = path_HE,
                                            IHC_img_dir = path_IHC,                                                     
                                           )

real_HE, real_IHC, img_name = train_data[4]
real_HE = torch.unsqueeze(real_HE,0)            

c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch.nn as nn
class Discriminator(nn.Module):
    def __init__(self, in_channels, hidden_dim, filter_groth, step_num ):
        super(Discriminator, self).__init__()
        
        self.conv = nn.Sequential(
                nn.ReflectionPad2d(in_channels), 
                nn.Conv2d(in_channels, hidden_dim, 2*in_channels+1),
                nn.ReLU(inplace=True),
                )

        channels = hidden_dim
        
        self.down = []
        for _ in range(step_num):
            out_channels = channels * filter_groth
            self.down += [
                nn.Conv2d(channels, out_channels, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(inplace=True),
            ]
            channels = out_channels
        self.down = nn.Sequential(*self.down)

    def forward(self, x):
        print(x.shape)
        x = self.conv(x)
        print(x.shape)
        x = self.down(x)
        print(x.shape)
        return x






c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\code\trans_models.py:238: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embedding = nn.Parameter(torch.tensor(get_positional_Embeddings((patch_size[0]**2)* chw[0] +1 , self.embedding_dim)))
c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\code\trans_models.py:158: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().deta

In [3]:
import conv_models

conv_gen = conv_models.Generator(in_channels= params['in_channels'],
                                        num_residual_blocks = params['num_resnet'],
                                        U_net_filter_groth = params['U_net_filter_groth'],
                                        U_net_step_num = params['U_net_step_num']
                                        )
conv_disc= conv_models.Discriminator(in_channels= params['in_channels'])

conv_gen.cuda()
conv_disc.cuda()

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): ZeroPad2d((1, 0, 1, 0))
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [4]:
import numpy as np
criterion_GAN = torch.nn.MSELoss().cuda()
Tensor = torch.cuda.FloatTensor

valid = Tensor(np.ones((real_HE.size(0)))) 
fake = Tensor(np.zeros((real_HE.size(0)))) 

fake_IHC = trans_gen(real_HE) 
disc_out = trans_disc(fake_IHC)
print(valid.shape)
print (disc_out.shape)
loss_gen_1= criterion_GAN(disc_out, valid) 
print(fake_IHC)
print(loss_gen_1)

loss_gen_2 =  criterion_GAN(fake_IHC, real_IHC)

torch.Size([1, 64])
torch.Size([1, 64])
torch.Size([1])
torch.Size([1, 64])
tensor([[[[ 1.3787e-01,  3.0926e-01,  2.1851e-01,  ...,  1.0902e+00,
            4.5783e-01, -1.8501e-01],
          [-1.1995e+00, -4.5390e-01,  2.0066e-01,  ...,  1.1940e+00,
           -6.7412e-01,  2.1906e+00],
          [ 9.6044e-01, -7.2076e-01, -5.1151e-01,  ...,  1.0378e+00,
           -1.2094e+00,  1.4771e+00],
          ...,
          [-1.5159e-01,  2.0645e-01, -1.6924e+00,  ...,  9.9420e-01,
           -3.3700e-01, -3.1481e-01],
          [-1.4030e+00, -9.9404e-01, -1.4305e-01,  ...,  8.9380e-01,
           -4.1937e-01,  1.0062e+00],
          [ 7.4775e-01, -6.5869e-01, -2.0023e+00,  ...,  1.7459e+00,
           -7.5956e-01,  1.9543e+00]],

         [[-1.6774e-03,  1.5214e+00, -2.1105e-01,  ...,  1.2275e+00,
           -7.2509e-01,  1.9181e+00],
          [-1.0436e+00, -1.2467e+00,  1.3783e-01,  ...,  1.4391e+00,
            5.2413e-01,  1.8759e+00],
          [ 9.5651e-01, -3.0956e-01,  6.8828e-01,  

c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\env\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\env\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([3, 256, 256])) that is different to the input size (torch.Size([1, 3, 256, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
